<a href="https://colab.research.google.com/github/georgialoukatou/childes-tagging/blob/main/sampling2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [ ]:
%%R
install.packages('devtools')

In [7]:
%%R
devtools::install_github("langcog/childesr")

R[write to console]: Skipping install of 'childesr' from a github remote, the SHA1 (fe949a75) has not changed since last install.
  Use `force = TRUE` to force installation



In [8]:
import pandas as pd
import spacy
import numpy as np
!pip install childespy
import childespy
from os import path

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



Reinstalling childesr version 0.2.1


In [9]:
NUM_BINS = 5
TOTAL_SAMPLES = 1000
MAX_POS_SAMPLES = 250

# need collection, language, corpus, and role if token_csv_name is none
COLLECTION = None
LANGUAGE = 'Italian'
CORPUS = 'Tonelli'
ROLE = None

UTTERANCE_CSV_NAME = None 
TOKEN_CSV_NAME = None

FULL_SAMPLED_TOKENS_CSV_NAME = 'sampled_full.csv'

In [10]:
if (TOKEN_CSV_NAME is not None and path.exists(TOKEN_CSV_NAME)):
    all_tokens = pd.read_csv(TOKEN_CSV_NAME, keep_default_na = False, index_col=0)
else:
    all_tokens = childespy.get_tokens(collection=COLLECTION, language=LANGUAGE, corpus=CORPUS, role=ROLE, token="%")


R[write to console]: Using current database version: '2020.1'.

R[write to console]: Getting data from 3 children in 1 corpus ...



In [22]:
all_tokens = childespy.get_tokens(corpus='Tonelli', token="%")

R[write to console]: Using current database version: '2020.1'.

R[write to console]: Getting data from 3 children in 1 corpus ...



In [23]:
print(all_tokens)

              id      gloss  ... transcript_id  utterance_id
1       28479330  guardiamo  ...         22559       7458988
2       28479331         se  ...         22559       7458988
3       28479332          è  ...         22559       7458988
4       28479333     uguale  ...         22559       7458988
5       28479334  guardiamo  ...         22559       7459010
...          ...        ...  ...           ...           ...
114066  28684816         no  ...         22568       7514843
114067  28684817    trenino  ...         22568       7514843
114068  28684818        via  ...         22568       7514844
114069  28684819         va  ...         22568       7514845
114070  28684820         va  ...         22568       7514845

[114070 rows x 28 columns]


In [25]:
token_copy = all_tokens.copy()
token_copy = token_copy[token_copy['part_of_speech'].astype(bool)] # omit the empty POS
token_copy = token_copy[token_copy['part_of_speech'].str.find(" ") == -1] # omit multiple-POS entries
token_copy = token_copy[~token_copy['gloss'].str.contains('xxx')]
token_copy = token_copy[~token_copy['gloss'].str.contains('yyy')]

def simplify_pos(pos):
    return pos[:pos.find(":")] if (pos.find(":") != -1 and pos.find(" ") == -1) else pos

def contraction_pos(part_of_speech, clitic, suffix):
    '''
    this function is very english-centric and childes-centric
    could incorporate another model, e.g. spacy, to find contractions
    '''
    if suffix == "dn POSS" or suffix == "dn AGT POSS":
        return part_of_speech + "+poss"
    if clitic == "":
        return part_of_speech
    else:
        return part_of_speech + "+" + clitic.split()[0]

simplified_pos_tokens = token_copy.copy()
simplified_pos_tokens['part_of_speech'] = token_copy['part_of_speech'].map(simplify_pos)
simplified_pos_tokens['part_of_speech'] = simplified_pos_tokens.apply(lambda x: contraction_pos(x['part_of_speech'], x['clitic'], x['suffix']), axis=1)
simplified_pos_tokens


,id,gloss,language,token_order,prefix,part_of_speech,stem,actual_phonology,model_phonology,suffix,num_morphemes,english,clitic,utterance_type,corpus_name,speaker_code,speaker_name,speaker_role,target_child_name,target_child_age,target_child_sex,collection_name,collection_id,corpus_id,speaker_id,target_child_id,transcript_id,utterance_id
1,28479330,guardiamo,ita,1,,vimp,guarda,,,1P IMP,3,look,,declarative,Tonelli,MOT,None,Mother,Elisa,23.131892,female,Romance,10,197,14407,14406,22559,7458988
2,28479331,se,ita,2,,conj,se,,,,1,,,declarative,Tonelli,MOT,None,Mother,Elisa,23.131892,female,Romance,10,197,14407,14406,22559,7458988
3,28479332,è,ita,3,,v,esse,,,3S PRES,3,be,,declarative,Tonelli,MOT,None,Mother,Elisa,23.131892,female,Romance,10,197,14407,14406,22559,7458988
4,28479333,uguale,ita,4,,adj,uguale,,,sg,2,,,declarative,Tonelli,MOT,None,Mother,Elisa,23.131892,female,Romance,10,197,14407,14406,22559,7458988
5,28479334,guardiamo,ita,1,,vimp,guarda,,,1P IMP,3,look,,declarative,Tonelli,CHI,Elisa,Target_Child,Elisa,23.131892,female,Romance,10,197,14406,14406,22559,7459010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114065,28684815,spingo,ita,9,,v,spinge,,,1S PRES,3,push,,declarative,Tonelli,MOT,None,Mother,Gregorio,20.723218,male,Romance,10,197,14409,14408,22568,7514842
114066,28684816,no,ita,1,,adv,no,,,,1,,,declarative,Tonelli,CHI,Gregorio,Target_Child,Gregorio,20.723218,male,Romance,10,197,14408,14408,22568,7514843
114067,28684817,trenino,ita,2,,n,treno,,,DIM m sg,4,,,declarative,Tonelli,CHI,Gregorio,Target_Child,Gregorio,20.723218,male,Romance,10,197,14408,14408,22568,7514843
114068,28684818,via,ita,1,,adv,via,,,,1,,,declarative,Tonelli,MOT,None,Mother,Gregorio,20.723218,male,Romance,10,197,14409,14408,22568,7514844


In [35]:
#pos_groupby = simplified_pos_tokens.groupby(['part_of_speech'])
#print(pos_groupby)

#gb = pos_groupby.groups

#for key, values in gb.items():
 #   print(key,values)

L2 Index(['27442', '41536'], dtype='object')
adj Index(['4', '8', '43', '45', '121', '124', '129', '135', '140', '146',
       ...
       '113642', '113745', '113747', '113758', '113789', '113795', '113843',
       '113859', '114037', '114047'],
      dtype='object', length=4272)
adv Index(['30', '32', '41', '52', '55', '64', '81', '92', '96', '101',
       ...
       '113992', '114024', '114025', '114026', '114042', '114044', '114056',
       '114058', '114066', '114068'],
      dtype='object', length=16236)
adv+pro Index(['580', '2762', '2998', '3389', '4130', '5282', '6022', '6124', '6869',
       '9388',
       ...
       '107847', '109300', '109495', '110093', '111581', '111631', '111953',
       '112786', '113196', '113270'],
      dtype='object', length=203)
art Index(['11', '36', '50', '53', '58', '60', '62', '67', '71', '77',
       ...
       '113940', '113967', '113979', '113981', '113993', '114000', '114006',
       '114028', '114030', '114060'],
      dtype='object', lengt

In [36]:
all_samples = pd.DataFrame()

def sample_from(pos_df, all_samples):
    # aggregate by type to count type frequency within POS
    types = pos_df.groupby(['gloss']).id.agg(np.size).reset_index()
    pos_df_with_frequency = pos_df.merge(types, on=['gloss'], suffixes=(None, "_frequency"))
    pos_df_with_frequency.rename(columns={'id_frequency': 'frequency'}, inplace=True)
    pos_df_with_frequency['log_frequency'] = np.nan
    pos_df_with_frequency['log_frequency'] = np.log(pos_df_with_frequency['frequency'])

    # assign frequency bin to each token
    token_bins = pd.cut(pos_df_with_frequency['log_frequency'], NUM_BINS, labels=list(range(NUM_BINS)))
    pos_df_with_bins = pos_df_with_frequency.join(token_bins, rsuffix="_bin")

    samples_per_bin = int(MAX_POS_SAMPLES/NUM_BINS)
    logfreq_sampled_tokens = pos_df_with_bins.groupby('log_frequency_bin').apply(lambda x: x.sample(n=samples_per_bin, replace=True) if len(x)>0 else x).reset_index(drop=True)
    logfreq_sampled_tokens.drop_duplicates(keep='first', inplace=True)
    all_samples = pd.concat([all_samples, logfreq_sampled_tokens])
    return all_samples

pos_groupby = simplified_pos_tokens.groupby(['part_of_speech'])
iterator = ["n", "v", "adj",  "adv",  "prep", "pro", "det", "prepart"] #"mod","part", "prepart added for italian"
for name in iterator:
    pos_df = pos_groupby.get_group(name) # dataframe with only the tokens of this part of speech
    all_samples = sample_from(pos_df, all_samples)

contraction_df = simplified_pos_tokens[simplified_pos_tokens['part_of_speech'].str.contains("\+")]
all_samples = sample_from(contraction_df, all_samples)

print(all_samples[["id", "gloss"]])


           id        gloss
0    28574144   cartoncino
1    28580837        verdi
2    28589106  termosifone
3    28539298       cugina
4    28610203   fiammiferi
..        ...          ...
245  28520300       eccola
246  28566852       eccola
247  28538989        delle
248  28601276       eccolo
249  28549704       eccolo

[1966 rows x 2 columns]


In [37]:
pos_groupby = simplified_pos_tokens.groupby(['part_of_speech'])
print(pos_groupby)
for key, item in pos_groupby:
  print(pos_groupby.get_group(key))

             id      gloss  ... transcript_id  utterance_id
27442  28551376  afterpick  ...         22575       7474080
41536  28565519    alright  ...         22585       7478591

[2 rows x 28 columns]
              id    gloss  ... transcript_id  utterance_id
4       28479333   uguale  ...         22559       7458988
8       28479337   uguale  ...         22559       7459010
43      28479372  morbido  ...         22559       7459092
45      28479374  morbido  ...         22559       7459105
121     28479450  morbido  ...         22559       7459287
...          ...      ...  ...           ...           ...
113795  28681593     vero  ...         22568       7514605
113843  28681641    bravo  ...         22568       7514757
113859  28681657     vero  ...         22568       7514761
114037  28684787    rotte  ...         22568       7514834
114047  28684797     vero  ...         22568       7514837

[4272 rows x 28 columns]
              id  gloss language  ...  target_child_id transcri

In [38]:
thousand_samples = all_samples.sample(n=TOTAL_SAMPLES).reset_index(drop=True)
thousand_samples

,id,gloss,language,token_order,prefix,part_of_speech,stem,actual_phonology,model_phonology,suffix,num_morphemes,english,clitic,utterance_type,corpus_name,speaker_code,speaker_name,speaker_role,target_child_name,target_child_age,target_child_sex,collection_name,collection_id,corpus_id,speaker_id,target_child_id,transcript_id,utterance_id,frequency,log_frequency,log_frequency_bin
0,28582986,da,ita,5,,prep,da,,,,1,at,,declarative,Tonelli,MOT,None,Mother,Marco,27.066264,male,Romance,10,197,14413,14412,22593,7484472,173,5.153292,3
1,28567731,coi,ita,1,,prepart,co,,,m pl,3,,,question,Tonelli,MOT,None,Mother,Marco,21.953223,male,Romance,10,197,14413,14412,22583,7478954,31,3.433987,3
2,28542409,tutta,ita,6,,det,tutto,,,f sg,3,all,,declarative,Tonelli,MOT,None,Mother,Marco,17.131769,male,Romance,10,197,14413,14412,22573,7473003,20,2.995732,2
3,28577323,nasconderle,ita,4,,inf+pro,nasconde,,,INF,3,,pro 3SP f,declarative,Tonelli,MOT,None,Mother,Marco,21.953223,male,Romance,10,197,14413,14412,22583,7485753,1,0.000000,0
4,28563710,cavalli,ita,1,,n,cavallo,,,m pl,3,horse,,declarative,Tonelli,CHI,Marco,Target_Child,Marco,19.624633,male,Romance,10,197,14412,14412,22578,7487551,74,4.304065,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,28556074,semplice,ita,3,,adj,semplice,,,sg,2,,,declarative,Tonelli,MOT,None,Mother,Marco,21.493254,male,Romance,10,197,14413,14412,22582,7471775,4,1.386294,1
996,28603401,chi,ita,1,,pro,chi,,,,1,who,,question,Tonelli,CHI,Marco,Target_Child,Marco,25.361917,male,Romance,10,197,14412,14412,22590,7490917,513,6.240276,4
997,28492751,questo,ita,8,,det,questo,,,m sg,3,this,,declarative,Tonelli,MOT,None,Mother,Elisa,22.131871,female,Romance,10,197,14407,14406,22558,7459697,345,5.843544,4
998,28598942,gliela,ita,8,,pro,gli,,,la,2,,,question,Tonelli,MOT,None,Mother,Marco,24.460461,male,Romance,10,197,14413,14412,22588,7486451,3,1.098612,0


In [39]:
if (UTTERANCE_CSV_NAME is not None and path.exists(UTTERANCE_CSV_NAME)):
    utterances = pd.read_csv(UTTERANCE_CSV_NAME, keep_default_na=False)
else:
    utterances = childespy.get_utterances(corpus=CORPUS)


R[write to console]: Using current database version: '2020.1'.

R[write to console]: Getting data from 3 children in 1 corpus ...



In [40]:
utterance_glosses = utterances.filter(['id', 'gloss'], axis=1)
samples_with_context = thousand_samples.merge(utterance_glosses, how='inner', left_on='utterance_id', right_on='id', suffixes=(None, '_utterance'))
samples_with_context.drop(samples_with_context.columns[samples_with_context.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
samples_with_context = samples_with_context.rename(columns={'gloss_utterance': 'utterance_gloss', 'id': 'token_id'})
samples_with_context = samples_with_context.drop(columns=['id_utterance', 'part_of_speech'])
samples_with_context = samples_with_context.merge(all_tokens[['id', 'part_of_speech']], how="left", left_on='token_id', right_on='id')
samples_with_context = samples_with_context.drop(columns=['id'])
samples_with_context


,token_id,gloss,language,token_order,prefix,stem,actual_phonology,model_phonology,suffix,num_morphemes,english,clitic,utterance_type,corpus_name,speaker_code,speaker_name,speaker_role,target_child_name,target_child_age,target_child_sex,collection_name,collection_id,corpus_id,speaker_id,target_child_id,transcript_id,utterance_id,frequency,log_frequency,log_frequency_bin,utterance_gloss,part_of_speech
0,28582986,da,ita,5,,da,,,,1,at,,declarative,Tonelli,MOT,None,Mother,Marco,27.066264,male,Romance,10,197,14413,14412,22593,7484472,173,5.153292,3,sì è abbastanza grande da farti da capanna,prep
1,28567731,coi,ita,1,,co,,,m pl,3,,,question,Tonelli,MOT,None,Mother,Marco,21.953223,male,Romance,10,197,14413,14412,22583,7478954,31,3.433987,3,coi denti,prepart
2,28542409,tutta,ita,6,,tutto,,,f sg,3,all,,declarative,Tonelli,MOT,None,Mother,Marco,17.131769,male,Romance,10,197,14413,14412,22573,7473003,20,2.995732,2,premi premi forte premi con tutta la mano,det
3,28577323,nasconderle,ita,4,,nasconde,,,INF,3,,pro 3SP f,declarative,Tonelli,MOT,None,Mother,Marco,21.953223,male,Romance,10,197,14413,14412,22583,7485753,1,0.000000,0,però possiamo anche nasconderle xxx,inf
4,28563710,cavalli,ita,1,,cavallo,,,m pl,3,horse,,declarative,Tonelli,CHI,Marco,Target_Child,Marco,19.624633,male,Romance,10,197,14412,14412,22578,7487551,74,4.304065,3,cavalli,n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,28556074,semplice,ita,3,,semplice,,,sg,2,,,declarative,Tonelli,MOT,None,Mother,Marco,21.493254,male,Romance,10,197,14413,14412,22582,7471775,4,1.386294,1,moto più semplice,adj
996,28603401,chi,ita,1,,chi,,,,1,who,,question,Tonelli,CHI,Marco,Target_Child,Marco,25.361917,male,Romance,10,197,14412,14412,22590,7490917,513,6.240276,4,chi è,pro:int
997,28492751,questo,ita,8,,questo,,,m sg,3,this,,declarative,Tonelli,MOT,None,Mother,Elisa,22.131871,female,Romance,10,197,14407,14406,22558,7459697,345,5.843544,4,la cucina è qui dove c' è questo signore qui,det
998,28598942,gliela,ita,8,,gli,,,la,2,,,question,Tonelli,MOT,None,Mother,Marco,24.460461,male,Romance,10,197,14413,14412,22588,7486451,3,1.098612,0,senti e a Lorena una telefonata non gliela fac...,pro:clit


In [41]:
samples_with_context.to_csv(FULL_SAMPLED_TOKENS_CSV_NAME)